In [6]:
import sys
sys.path.append('/home/maria/NeuroGarage/HuggingMouse_local')
from dotenv import load_dotenv
load_dotenv()
from HuggingMouse import make_embeddings
from pathlib import Path
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import numpy as np
import torch
import pickle
import os


/home/maria/NeuroGarage/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
session_A = 501704220  # This is three session A
session_B = 501559087
session_C = 501474098
allen_cache_path = os.environ.get('HGMS_ALLEN_CACHE_PATH')
boc = BrainObservatoryCache(manifest_file=str(
        Path(allen_cache_path) / Path('brain_observatory_manifest.json')))
raw_data_dct = {}
movie_one_dataset = boc.get_ophys_experiment_data(session_A)
raw_data_dct['natural_movie_one'] = movie_one_dataset.get_stimulus_template(
        'natural_movie_one')
natural_stimuli = boc.get_ophys_experiment_data(session_B)
raw_data_dct['natural_scenes'] = natural_stimuli.get_stimulus_template(
        'natural_scenes')
boc.get_ophys_experiment_data(session_C)

In [8]:
raw_data_dct['natural_movie_one'].shape

(900, 304, 608)

In [9]:
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
inputs = processor(images=image, return_tensors="pt")

outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

/home/maria/NeuroGarage/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
import sys
sys.path.append('/home/maria/NeuroGarage/')
from dotenv import load_dotenv
load_dotenv()
from HuggingMouse_local.src.HuggingMouse import make_embeddings
from transformers import ViTImageProcessor, ViTModel

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
embs=make_embeddings.MakeEmbeddings(processor,model)
embs.execute()

/home/maria/NeuroGarage/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(900, 304, 608)
Time taken for embedding one movie:  102.78213286399841
(900, 304, 608)
Time taken for embedding one movie:  109.43440008163452
(3600, 304, 608)
Time taken for embedding one movie:  404.09574007987976
(118, 918, 1174)
Time taken for embedding one movie:  18.679101943969727


{'natural_movie_one': array([[ 0.15474109, -0.21664083,  0.28223929, ..., -0.09267968,
          0.28659159, -0.24431331],
        [ 0.29176757, -0.07541179, -0.21240199, ..., -0.17095642,
         -0.45944124, -0.55567753],
        [ 0.31295866, -0.11137368, -0.33041185, ..., -0.1862639 ,
         -0.48867032, -0.5173654 ],
        ...,
        [ 0.49249977, -0.66381383,  0.2804504 , ...,  0.36285573,
          0.0133081 , -0.32655248],
        [ 0.54520386, -0.64170301,  0.32991105, ...,  0.67677408,
         -0.2688832 ,  0.11149126],
        [ 0.55117595, -0.71902919, -0.13872029, ...,  0.3683044 ,
         -0.08161116,  0.10802519]]),
 'natural_movie_two': array([[ 0.50542468, -0.74632651, -0.09666888, ...,  0.26893696,
          0.00080879, -0.0451454 ],
        [ 0.32269448, -0.7255069 ,  0.11043712, ...,  0.49895117,
         -0.14799875,  0.08729225],
        [ 0.3548902 , -0.65193462,  0.13763492, ...,  0.56592023,
         -0.20311958,  0.16430445],
        ...,
        [-0.

In [10]:
with open('/home/maria/Documents/HuggingMouseData/TransformerEmbeddings/google_vit-base-patch16-224-in21k_embeddings.pkl', 'rb') as file:
    embs = pickle.load(file)


In [4]:
embs['natural_scenes'].shape

(118, 768)

In [11]:
embs['natural_movie_one'].shape

(900, 768)

In [12]:
import numpy as np

# Assuming natural_movie_one is your original array with shape (900, 768)
natural_movie_one = embs['natural_movie_one'] # Replace this with your actual data

# Generate a random projection matrix of shape (768, 768)
random_projection_matrix = np.random.randn(768, 768)

# Perform the random projection
projected_data = np.dot(natural_movie_one, random_projection_matrix)

# The shape of projected_data should be (900, 768)
print("Shape of the projected data:", projected_data.shape)


Shape of the projected data: (900, 768)


In [18]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt#
%matplotlib widget

pca = PCA(n_components=3)
projected_data_3d = pca.fit_transform(natural_movie_one)

# Create a 3D plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot of the 3D projected data
ax.scatter(projected_data_3d[:, 0], projected_data_3d[:, 1], projected_data_3d[:, 2], c='blue', s=50, alpha=0.6)

# Label the axes
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')

# Set the title
ax.set_title('3D PCA Projection of Natural Movie Data')

# Show the plot
plt.show()



RuntimeError: 'widget is not a recognised GUI loop or backend name